Based on [this langchain pdf tutorial](https://python.langchain.com/docs/tutorials/pdf_qa/), combined with [this integration guide for langfuse & langchain](https://langfuse.com/docs/integrations/langchain/tracing).

A hosted version [on kaggle here](https://www.kaggle.com/code/sarincasm/pdf-qna/edit)


In [ ]:
%pip install -qU pypdf langchain_community langfuse openai langchain_openai

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "/kaggle/input/nike-annual-sec-report/414759-1-_5_Nike-NPS-Combo_Form-10-K_WR.pdf"
loader = PyPDFLoader(file_path)

docs = loader.load()

print(len(docs))

In [ ]:
print(docs[0].page_content)
print(docs[0].metadata)

In [ ]:
from langfuse.openai import openai
openai.langfuse_auth_check()


In [ ]:
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = InMemoryVectorStore.from_documents(
    documents=splits, embedding=OpenAIEmbeddings()
)

retriever = vectorstore.as_retriever()

In [ ]:
len(docs)

In [ ]:
from langfuse.callback import CallbackHandler
langfuse_handler = CallbackHandler()

In [ ]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o")

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

results = rag_chain.invoke({"input": "What was Nike's revenue in 2023?"}, config={"callbacks": [langfuse_handler]})

results